# 감염병 CDM 데이터 품질 보고서

In [114]:
# 환경 세팅
import pandas as pd
import os
import plotly.graph_objects as go
from plotly.subplots import make_subplots
cdm_path = "F:/01.감염병데이터베이스/data/2024_infectious_CDM"

provider 직군 분포 조사

In [25]:
table_name = "provider"
provider = pd.read_csv(os.path.join(cdm_path, table_name + ".csv"))
provider = provider.groupby("specialty_source_value_name").size()
provider = provider.sort_values(ascending=False)

provider=provider[:20]
fig = go.Figure(
    # Data 입력
    data = [go.Bar(x = provider.index, y=provider.values)]
)
fig.update_layout(title_text='provider 직군별 분포 Top 20', xaxis_title='specialty_source_value_name', yaxis_title='Count')

# 그래프 표시
fig.show()

환자 성별 및 나이 분포

In [103]:
table_name = "person"
person = pd.read_csv(os.path.join(cdm_path, table_name + ".csv"))

gender_counts = person["gender_source_value"].value_counts()
total_count = gender_counts.sum()
gender_percentages = (gender_counts / total_count * 100),round(2)

colors = ["ef553b", "636efa"]
labels = ["Male", "Female"]  # 변경하고 싶은 라벨 이름

label_map = {'M': 'Male', 'F': 'Female'}  # 실제 코드에 따라 수정 필요
mapped_labels = [label_map[label] for label in gender_counts.index]

# 도넛 차트 생성
fig = go.Figure(data = [go.Pie(labels=mapped_labels, values=gender_counts.values,
                               hole=0.4, # 중앙의 구멍 크기
                               textinfo="label+value+percent", # 라벨과 비율 정보 표시
                               texttemplate="%{value} <br> (%{percent})",
                               insidetextorientation='radial', # 텍스트 방향
                               marker_colors = colors # 색상 설정
                               )])

# 도넛 차트 중앙에 전체 명수 표시
fig.update_traces(textposition='inside', textinfo='none')
fig.update_layout(
    title_text="성별 분포",
    annotations=[{
        "font": {"size": 15},
        "showarrow": False,
        "text": f"Total: {total_count}", # 중앙에 표시할 텍스트
        "x": 0.5,
        "y": 0.5
    }]
)

# 그래프 표시
fig.show()

In [101]:
# 연도별 빈도 계산
year_counts = person["year_of_birth"].value_counts().sort_index()

fig = go.Figure(
    data = [go.Bar(x=year_counts.index,
                   y=year_counts.values,
                   marker=dict(line=dict(width=0))
                )])
fig.update_layout(title_text='첫 방문시 나이 분포', xaxis_title='year_of_birth', yaxis_title='People',
                #   paper_bgcolor="white", # 전체 배경색 설정
                  plot_bgcolor="white" # 플롯 배경색 설정
                  )
# X축 설정 - 눈금 표시, 그리드는 제거
fig.update_xaxes(
    showline=True,  # X축 선 표시
    linewidth=2,  # X축 선의 너비
    linecolor='black',  # X축 선의 색상
    showgrid=False,  # 그리드는 표시하지 않음
    ticks='outside',  # 눈금 표시 위치
    tickwidth=2,  # 눈금의 너비
    tickcolor='black',  # 눈금의 색상
    ticklen=6  # 눈금의 길이
)

# Y축 설정 - 눈금 표시, 그리드는 제거
fig.update_yaxes(
    showline=True,  # Y축 선 표시
    linewidth=2,  # Y축 선의 너비
    linecolor='black',  # Y축 선의 색상
    showgrid=False,  # 그리드는 표시하지 않음
    ticks='outside',  # 눈금 표시 위치
    tickwidth=2,  # 눈금의 너비
    tickcolor='black',  # 눈금의 색상
    ticklen=6  # 눈금의 길이
)

# 그래프 제목 및 레이아웃 설정
fig.update_layout(
    title='year of birth distribution',
    xaxis_title='year of birth',
    yaxis_title='People',
    plot_bgcolor='white',  # 배경색을 하얀색으로 설정
)

fig.show()

연도별 데이터 분포 표시

In [154]:
# 테이블 불러오기
dataframes = []
table_name = "visit_occurrence"
visit = pd.read_csv(os.path.join(cdm_path, table_name + ".csv"))
visit["date"] = pd.to_datetime(visit["visit_start_date"])
# 날짜 형식 변환: 'YYYY-MM' 형식으로 변환합니다.
visit['year_month'] = visit['date'].dt.to_period('M')
# 그룹화: 연도와 월별로 그룹화하여 레코드의 개수를 계산합니다.
visit_summary = visit.groupby('year_month').size().reset_index(name='records')
visit_summary['category'] = table_name
dataframes.append(visit_summary)


table_name = "condition_occurrence"
condition = pd.read_csv(os.path.join(cdm_path, table_name + ".csv"))
condition["date"] = pd.to_datetime(condition["condition_start_date"])
# 날짜 형식 변환: 'YYYY-MM' 형식으로 변환합니다.
condition['year_month'] = condition['date'].dt.to_period('M')
# 그룹화: 연도와 월별로 그룹화하여 레코드의 개수를 계산합니다.
condition_data = condition.groupby('year_month').size().reset_index(name='records')
condition_data['category'] = table_name
dataframes.append(condition_data)

table_name = "drug_exposure"
drug = pd.read_csv(os.path.join(cdm_path, table_name + ".csv"))
drug["date"] = pd.to_datetime(drug["drug_exposure_start_date"])
# 날짜 형식 변환: 'YYYY-MM' 형식으로 변환합니다.
drug['year_month'] = drug['date'].dt.to_period('M')
# 그룹화: 연도와 월별로 그룹화하여 레코드의 개수를 계산합니다.
drug_data = drug.groupby('year_month').size().reset_index(name='records')
drug_data['category'] = table_name
dataframes.append(drug_data)

table_name = "measurement"
measurement = pd.read_csv(os.path.join(cdm_path, table_name + ".csv"))
measurement["date"] = pd.to_datetime(measurement["measurement_date"])
# 날짜 형식 변환: 'YYYY-MM' 형식으로 변환합니다.
measurement['year_month'] = measurement['date'].dt.to_period('M')
# 그룹화: 연도와 월별로 그룹화하여 레코드의 개수를 계산합니다.
measurement_data = measurement.groupby('year_month').size().reset_index(name='records')
measurement_data['category'] = table_name
dataframes.append(measurement_data)

table_name = "procedure_occurrence"
procedure = pd.read_csv(os.path.join(cdm_path, table_name + ".csv"))
procedure["date"] = pd.to_datetime(procedure["procedure_date"])
# 날짜 형식 변환: 'YYYY-MM' 형식으로 변환합니다.
procedure['year_month'] = procedure['date'].dt.to_period('M')
# 그룹화: 연도와 월별로 그룹화하여 레코드의 개수를 계산합니다.
procedure_data = procedure.groupby('year_month').size().reset_index(name='records')
procedure_data['category'] = table_name
dataframes.append(procedure_data)

all_data = pd.concat(dataframes)

fig = go.Figure()
categories = all_data["category"].unique()
for category in categories:
    category_data = all_data[all_data['category'] == category]
    fig.add_trace(go.Scatter(x = category_data["year_month"].astype(str), y=category_data["records"], name=category, 
                          mode="lines"))

# X축 설정 - 눈금 표시, 그리드는 제거
fig.update_xaxes(
    showline=True,  # X축 선 표시
    linewidth=2,  # X축 선의 너비
    linecolor='black',  # X축 선의 색상
    showgrid=False,  # 그리드는 표시하지 않음
    ticks='outside',  # 눈금 표시 위치
    tickwidth=2,  # 눈금의 너비
    tickcolor='black',  # 눈금의 색상
    ticklen=6  # 눈금의 길이
)

# Y축 설정 - 눈금 표시, 그리드는 제거
fig.update_yaxes(
    showline=True,  # Y축 선 표시
    linewidth=2,  # Y축 선의 너비
    linecolor='black',  # Y축 선의 색상
    showgrid=False,  # 그리드는 표시하지 않음
    ticks='outside',  # 눈금 표시 위치
    tickwidth=2,  # 눈금의 너비
    tickcolor='black',  # 눈금의 색상
    ticklen=6,  # 눈금의 길이
    range=[0,max(all_data["records"])*1.05]
)

fig.update_layout(
    title="Total Rows over Time by Category",
    xaxis_title="Year",
    yaxis_title="# of Records",
    plot_bgcolor="white"
)

fig.show()


C:\Users\sangil\AppData\Local\Temp\ipykernel_29976\748649559.py:25: DtypeWarning:

Columns (38,39) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\sangil\AppData\Local\Temp\ipykernel_29976\748649559.py:35: DtypeWarning:

Columns (13,15,19,24,26,29,30,33,37,38,39) have mixed types. Specify dtype option on import or set low_memory=False.

C:\Users\sangil\AppData\Local\Temp\ipykernel_29976\748649559.py:45: DtypeWarning:

Columns (11,23,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.

